In [ ]:
pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
from datasets import load_dataset

dataset = load_dataset("samaxr/code-summary-java")
print(dataset)
train_data = dataset["train"]
subset = train_data.select(range(500))  # Select first 200 indices

# for item in subset:
#     code = item["code"]
#     summary = item["summary"]
    # ... (process code and summary)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/285670 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/31741 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/79352 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['code', 'summary'],
        num_rows: 285670
    })
    validation: Dataset({
        features: ['code', 'summary'],
        num_rows: 31741
    })
    test: Dataset({
        features: ['code', 'summary'],
        num_rows: 79352
    })
})


In [ ]:
from transformers import BertTokenizer
import concurrent.futures

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define function to preprocess a batch of data
def preprocess_batch(item):
    processed_batch = []
    code = item['code']
    summary = item['summary']

    # Tokenize code and summary
    code_tokens = tokenizer.encode(code, add_special_tokens=True, max_length=512, truncation=True, padding='max_length')
    summary_tokens = tokenizer.encode(summary, add_special_tokens=True, max_length=128, truncation=True, padding='max_length')

    # Add processed data to list
    processed_batch.append({
        'code_ids': code_tokens,
        'code_mask': [1] * len(code_tokens),
        'summary_ids': summary_tokens,
        'summary_mask': [1] * len(summary_tokens)
    })

    return processed_batch


# Define function to preprocess data in batches using multiprocessing
def preprocess_data_multiprocessed(data, batch_size):
    processed_data = []
    with concurrent.futures.ProcessPoolExecutor() as executor:
        # data_list = list(data)
        #train_data = data["train"]

        # Divide the data into batches
        batches = [data[i:i+batch_size] for i in range(0, len(data), batch_size)]
        # print(type(batches))
        # for item in batches:
        #   print(item)
        #   print(len(item['code']))

        # Preprocess each batch using multiple processes
        for processed_batch in executor.map(preprocess_batch, batches):
            processed_data.extend(processed_batch)
    return processed_data


# Set batch size
batch_size = 16

# Preprocess data in batches
# print(type(subset))
# print(subset.select(range(50)))
processed_data = preprocess_data_multiprocessed(train_data, batch_size)

print("Processed")

#print(processed_data)
# Now you can use 'processed_data' to train your code summarization model

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Processed


In [ ]:
pip install sacrebleu rouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 1.3 MB/s eta 0:00:00


In [ ]:
import concurrent.futures
from collections import Counter
from transformers import BertTokenizer
from datasets import load_dataset

# Load the pre-trained tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load the dataset
dataset = load_dataset("samaxr/code-summary-java")
train_data = dataset["train"]
subset = train_data.select(range(500))  # Select first 500 indices

# Define function to preprocess a single data item
def preprocess_item(item):
    processed_item = {}
    code = item['code']
    summary = item['summary']

    # Tokenize code and summary
    code_tokens = tokenizer.encode(code, add_special_tokens=True, max_length=512, truncation=True, padding='max_length')
    summary_tokens = tokenizer.encode(summary, add_special_tokens=True, max_length=128, truncation=True, padding='max_length')

    # Add processed data to dictionary
    processed_item['code_ids'] = code_tokens
    processed_item['code_mask'] = [1] * len(code_tokens)
    processed_item['summary_ids'] = summary_tokens
    processed_item['summary_mask'] = [1] * len(summary_tokens)

    return processed_item

# Define function to preprocess data in batches using multiprocessing
def preprocess_data_multiprocessed(data, batch_size):
    processed_data = []
    with concurrent.futures.ProcessPoolExecutor() as executor:
        # Preprocess each data item using multiple processes
        for processed_batch in executor.map(preprocess_item, data):
            processed_data.append(processed_batch)
    return processed_data

# Define function to generate n-grams from a given text
def generate_ngrams(text, n):
    tokens = [word.lower() for word in text.split()]  # Tokenize and lowercase words
    n_grams = []
    for i in range(len(tokens) - n + 1):
        n_grams.append(' '.join(tokens[i:i+n]))  # Create n-gram string
    return n_grams  # Return a list of n-grams

# Define function to calculate ROUGE-N recall score
def calculate_ROUGE(generated_summary, reference_summary, n):
    generated_ngrams = generate_ngrams(generated_summary, n)
    reference_ngrams = generate_ngrams(reference_summary, n)

    # Count the occurrences of n-grams in the generated summary
    gen_ngram_counts = Counter(generated_ngrams)
    ref_ngram_counts = Counter(reference_ngrams)

    # Compute the intersection and union of n-grams
    intersection = sum((gen_ngram_counts & ref_ngram_counts).values())
    union = sum((gen_ngram_counts | ref_ngram_counts).values())

    # Calculate recall score
    recall_score = intersection / union if union > 0 else 0

    return recall_score

# Preprocess the dataset
processed_dataset = preprocess_data_multiprocessed(subset, batch_size=1)




In [ ]:

from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset
import torch
import concurrent.futures
from tqdm import tqdm
import os

# Install the datasets library if not already installed
!pip install datasets

# Import load_dataset from datasets
from datasets import load_dataset

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Load the dataset
dataset = load_dataset("samaxr/code-summary-java")
train_data = dataset["train"]
subset = train_data.select(range(50))  # Select first 50 indices

# Define your dataset class
class CodeSummaryDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Define your model class
class CodeSummaryModel(torch.nn.Module):
    def __init__(self):
        super(CodeSummaryModel, self).__init__()
        self.bert = BertForSequenceClassification.from_pretrained("bert-base-uncased")

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        return logits

# Define function to preprocess a batch of data
def preprocess_batch(item):
    processed_batch = []
    code = item['code']
    summary = item['summary']

    # Tokenize code and summary
    code_tokens = tokenizer.encode(code, add_special_tokens=True, max_length=512, truncation=True, padding='max_length')
    summary_tokens = tokenizer.encode(summary, add_special_tokens=True, max_length=128, truncation=True, padding='max_length')

    # Add processed data to list
    processed_batch.append({
        'input_ids': code_tokens,
        'attention_mask': [1] * len(code_tokens),
        'labels': 0,  # Assuming this is for binary classification
        'summary_ids': summary_tokens,
        'summary_mask': [1] * len(summary_tokens)
    })

    return processed_batch

# Define function to preprocess data in batches using multiprocessing
def preprocess_data_multiprocessed(data, batch_size):
    processed_data = []
    with concurrent.futures.ProcessPoolExecutor() as executor:
        # Divide the data into batches
        batches = [data[i:i+batch_size] for i in range(0, len(data), batch_size)]

        # Preprocess each batch using multiple processes
        for processed_batch in executor.map(preprocess_batch, batches):
            processed_data.extend(processed_batch)
    return processed_data

# Set batch size
batch_size = 16

# Preprocess data in batches
processed_data = preprocess_data_multiprocessed(subset, batch_size)

# Convert preprocessed data to DataLoader
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return {key: torch.tensor(val) for key, val in self.data[idx].items()}

custom_dataset = CustomDataset(processed_data)
train_dataloader = DataLoader(custom_dataset, batch_size=batch_size)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set the number of epochs
epochs = 10  # Increased to 10 epochs

# Instantiate your model
model = CodeSummaryModel()
model.to(device)

# Define optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=2e-5)
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Define your training loop
def train(model, train_dataloader, optimizer, scheduler, device):
    model.train()
    total_loss = 0.0

    for batch in tqdm(train_dataloader, desc="Training"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        logits = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = torch.nn.functional.cross_entropy(logits.view(-1, 2), labels)  # Assuming binary classification
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Gradient clipping
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_dataloader)
    return avg_train_loss

# Train your model
for epoch in range(epochs):
    avg_train_loss = train(model, train_dataloader, optimizer, scheduler, device)
    print(f"Epoch {epoch+1}/{epochs}, Average Training Loss: {avg_train_loss:.4f}")

# Specify the directory to save the model
save_dir = '/content/model_directory'

# Create the directory if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

# Save the model
torch.save(model.state_dict(), os.path.join(save_dir, 'model.pth'))

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Training: 100%|██████████| 1/1 [00:47<00:00, 47.64s/it]


Epoch 1/10, Average Training Loss: 0.7567


Training: 100%|██████████| 1/1 [00:28<00:00, 28.22s/it]


Epoch 2/10, Average Training Loss: 0.4634


...

Epoch 8/10, Average Training Loss: 0.2040


Training: 100%|██████████| 1/1 [00:25<00:00, 25.79s/it]


Epoch 9/10, Average Training Loss: 0.1609


Training: 100%|██████████| 1/1 [00:25<00:00, 25.88s/it]


Epoch 10/10, Average Training Loss: 0.1515


In [ ]:
tokenized_code = tokenizer(["public class Main {\n    public static void main(String[] args) {\n        int num1 = 5;\n        int num2 = 10;\n        int sum = num1 + num2;\n        System.out.println(\"Sum: \" + sum);\n    }\n", "class Test{   public static void main(String []args) {  System.out.println(\"My First Java Program.\");  }};"])

print(tokenized_code)

{'input_ids': [[101, 2270, 2465, 2364, 1063, 2270, 10763, 11675, 2364, 1006, 5164, 1031, 1033, 12098, 5620, 1007, 1063, 20014, 16371, 2213, 2487, 1027, 1019, 1025, 20014, 16371, 2213, 2475, 1027, 2184, 1025, 20014, 7680, 1027, 16371, 2213, 2487, 1009, 16371, 2213, 2475, 1025, 2291, 1012, 2041, 1012, 6140, 19666, 1006, 1000, 7680, 1024, 1000, 1009, 7680, 1007, 1025, 1065, 102], [101, 2465, 3231, 1063, 2270, 10763, 11675, 2364, 1006, 5164, 1031, 1033, 12098, 5620, 1007, 1063, 2291, 1012, 2041, 1012, 6140, 19666, 1006, 1000, 2026, 2034, 9262, 2565, 1012, 1000, 1007, 1025, 1065, 1065, 1025, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
!pip install rouge_score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=9eb0c22df6718c49fe2cccde5ccd9a5de30be77627c7b2081b6ce701cd2e6db3
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
from datasets import load_metric
import concurrent.futures
from collections import Counter
from transformers import BertTokenizer
from datasets import load_dataset
# Load ROUGE metric
rouge_metric = load_metric("rouge")

# Calculate ROUGE-1 scores for each example
scores = []
for item in subset:
    code = item["code"]
    summary = item["summary"]
    rouge_output = rouge_metric.compute(predictions=[summary], references=[code])
    rouge_1_score = rouge_output["rouge1"].mid.fmeasure
    scores.append(rouge_1_score)

# Calculate the mean ROUGE-1 score
mean_rouge_1_score = sum(scores) / len(scores)
print("Mean ROUGE-1 Score:", mean_rouge_1_score)


<ipython-input-8-01085933c791>:7: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Mean ROUGE-1 Score: 0.07559574429231343


In [ ]:
def predict_summary(code, model, tokenizer, device, max_length=128):
    model.eval()
    with torch.no_grad():
        encoding = tokenizer(code, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
        input_ids = encoding['input_ids'].to(device)
        attention_mask = encoding['attention_mask'].to(device)

        # outputs = model.generate(input_ids, max_new_tokens=100, do_sample=False)


        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        print("OUTPUT:" , outputs)

        # Assuming logits are directly returned by the model
        logits = outputs

        predicted_summary_ids = torch.argmax(logits, dim=-1)
        print("Predicted IDS : ", predicted_summary_ids)

    # Decode the predicted summary
    predicted_summary = tokenizer.decode(predicted_summary_ids.squeeze(), skip_special_tokens=True)
    return predicted_summary


In [ ]:
# Example usage
test_code = "public class Main {\n    public static void main(String[] args) {\n        int num1 = 5;\n        int num2 = 10;\n        int sum = num1 + num2;\n        System.out.println(\"Sum: \" + sum);\n    }\n}"
predicted_summary = predict_summary(test_code, model, tokenizer, device)
print("Original Code:")
print(test_code)
print("\nPredicted Summary:")
print(predicted_summary)


OUTPUT: tensor([[ 0.1840, -0.1738]])
Predicted IDS :  tensor([0])
Original Code:
public class Main {
    public static void main(String[] args) {
        int num1 = 5;
        int num2 = 10;
        int sum = num1 + num2;
        System.out.println("Sum: " + sum);
    }
}

Predicted Summary:
[ P A D ]


In [ ]:
test_code = "class MergeSort { void merge(int arr[], int l, int m, int r) { int n1 = m - l + 1; int n2 = r - m; int L[] = new int[n1]; int R[] = new int[n2]; for (int i = 0; i < n1; ++i) L[i] = arr[l + i]; for (int j = 0; j < n2; ++j) R[j] = arr[m + 1 + j]; int i = 0, j = 0; int k = l; while (i < n1 && j < n2) { if (L[i] <= R[j]) { arr[k] = L[i]; i++; } else { arr[k] = R[j]; j++; } k++; } while (i < n1) { arr[k] = L[i]; i++; k++; } while (j < n2) { arr[k] = R[j]; j++; k++; } } void sort(int arr[], int l, int r) { if (l < r) { int m = (l + r) / 2; sort(arr, l, m); sort(arr, m + 1, r); merge(arr, l, m, r); } } static void printArray(int arr[]) { int n = arr.length; for (int i = 0; i < n; ++i) System.out.print(arr[i] + " "); System.out.println(); } public static void main(String args[]) { int arr[] = { 12, 11, 13, 5, 6, 7 }; printArray(arr); MergeSort ob = new MergeSort(); ob.sort(arr, 0, arr.length - 1); printArray(arr); } }"
predicted_summary = predict_summary(test_code, model, tokenizer, device)
print("Original Code:")
print(test_code)
print("\nPredicted Summary:")
print(predicted_summary)

OUTPUT: tensor([[0.3574, 0.1308]])
Predicted IDS :  tensor([0])
Original Code:
class MergeSort { void merge(int arr[], int l, int m, int r) { int n1 = m - l + 1; int n2 = r - m; int L[] = new int[n1]; int R[] = new int[n2]; for (int i = 0; i < n1; ++i) L[i] = arr[l + i]; for (int j = 0; j < n2; ++j) R[j] = arr[m + 1 + j]; int i = 0, j = 0; int k = l; while (i < n1 && j < n2) { if (L[i] <= R[j]) { arr[k] = L[i]; i++; } else { arr[k] = R[j]; j++; } k++; } while (i < n1) { arr[k] = L[i]; i++; k++; } while (j < n2) { arr[k] = R[j]; j++; k++; } } void sort(int arr[], int l, int r) { if (l < r) { int m = (l + r) / 2; sort(arr, l, m); sort(arr, m + 1, r); merge(arr, l, m, r); } } static void printArray(int arr[]) { int n = arr.length; for (int i = 0; i < n; ++i) System.out.print(arr[i] + ); System.out.println(); } public static void main(String args[]) { int arr[] = { 12, 11, 13, 5, 6, 7 }; printArray(arr); MergeSort ob = new MergeSort(); ob.sort(arr, 0, arr.length - 1); printArray(arr); } }


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("/content/model_directory/model.pth")

# Input code snippet
input_code = "public class Main {\n    public static void main(String[] args) {\n        int num1 = 5;\n        int num2 = 10;\n        int sum = num1 + num2;\n        System.out.println(\"Sum: \" + sum);\n    }\n}"

# Tokenize the input code snippet
tokenized_input = tokenizer(input_code, return_tensors="pt", max_length=512, truncation=True, padding="max_length")

# Generate summary
with torch.no_grad():
    outputs = model(**tokenized_input)

# Get the predicted class (assuming binary classification)
predicted_class = torch.argmax(outputs.logits, dim=-1).item()

# Print the predicted class
print("Predicted class:", predicted_class)


OSError: Incorrect path_or_model_id: '/content/model_directory/model.pth'. Please provide either the path to a local folder or the repo_id of a model on the Hub.